In [7]:
def custom_train_test_split(df, ratio=0.7, split=True):
    
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)


    train = df[df['userID'].isin(user_ids)]
    test = df[df['userID'].isin(user_ids) == False]

    #test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    return train, test

In [25]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import random

FEATS = [
   'KnowledgeTag', 'month', 'hour', 'week', 'elapsed', #'elapsed_cate',
   'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2',
   'testId0', 'testId1',# 'test0_mean', 'test0_std', 'test1_mean',
   #'test1_std', 'tag_mean', 'tag_std', 'ass0_mean', 'ass0_std',
   #'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std',
'as0_as1', 'as0_as2', 'as1_as2', 'assessmentItemID', 'week_hour', 
'KnowledgeTag2', 'assessmentItemID11', 'week_hour2']

cate = ['KnowledgeTag', 'month', 'hour', 'week', 'testId0', 'testId1', #'elapsed_cate'
       'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2',
        'as0_as1', 'as0_as2', 'as1_as2', 'assessmentItemID', 'week_hour',
       'KnowledgeTag2', 'assessmentItemID11', 'week_hour2']


# cate = ['KnowledgeTag', 'month', 'hour', 'week', 'week_hour', 'elapsed_cate',

# 'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2',]

# conti = ['tag_mean', 'tag_std', 'ass0_mean', 'ass0_std', 'elapsed',

# 'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std', 'user_total_answer', 'solve_order']

#FEATS = cate + conti
df = pd.read_csv('../../data/elo.csv')

df['KnowledgeTag2'] = df['KnowledgeTag']
df['assessmentItemID11'] = df['assessmentItemID1']
df['week_hour2'] = df['week_hour']

for i in cate:
    df[i] = df[i].astype('category')

X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(df[FEATS], df['answerCode'], test_size=0.2, shuffle=True, random_state=42)

# train, test = custom_train_test_split(df)

# y_train_cat = train['answerCode']
# X_train_cat = train.drop(['answerCode'], axis=1)[FEATS]

# y_test_cat = test['answerCode']
# X_test_cat = test.drop(['answerCode'], axis=1)[FEATS]

In [23]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np

catboost_cl = CatBoostClassifier(cat_features=cate, n_estimators=10000, eval_metric ='AUC',
                                use_best_model=False, learning_rate=0.015)

catboost_cl.fit(df[FEATS], df['answerCode'], verbose=100, eval_set=(X_test_cat, y_test_cat))

preds = catboost_cl.predict_proba(X_test_cat)[:,1]
acc = accuracy_score(y_test_cat, np.where(preds >= 0.5, 1, 0))
auc = roc_auc_score(y_test_cat, preds)

print(f'VALID AUC : {auc} ACC : {acc}\n')

0:	test: 0.7858805	best: 0.7858805 (0)	total: 2.82s	remaining: 7h 50m 35s
100:	test: 0.8111706	best: 0.8111706 (100)	total: 3m 7s	remaining: 5h 6m 57s
200:	test: 0.8315080	best: 0.8315080 (200)	total: 7m 47s	remaining: 6h 19m 42s
300:	test: 0.8506451	best: 0.8506451 (300)	total: 12m 23s	remaining: 6h 39m 15s
400:	test: 0.8633003	best: 0.8633003 (400)	total: 16m 46s	remaining: 6h 41m 25s
500:	test: 0.8589338	best: 0.8652977 (448)	total: 21m 12s	remaining: 6h 42m 9s
600:	test: 0.8502585	best: 0.8652977 (448)	total: 25m 35s	remaining: 6h 40m 8s
700:	test: 0.8455412	best: 0.8652977 (448)	total: 29m 52s	remaining: 6h 36m 14s
800:	test: 0.8438544	best: 0.8652977 (448)	total: 33m 54s	remaining: 6h 29m 22s
900:	test: 0.8440323	best: 0.8652977 (448)	total: 37m 58s	remaining: 6h 23m 28s
1000:	test: 0.8448731	best: 0.8652977 (448)	total: 41m 59s	remaining: 6h 17m 33s
1100:	test: 0.8448979	best: 0.8652977 (448)	total: 46m 12s	remaining: 6h 13m 29s
1200:	test: 0.8443913	best: 0.8652977 (448)	total:

KeyboardInterrupt: 

In [11]:
sub = pd.read_csv('../../data/infer.csv')

for i in cate:
    sub[i] = sub[i].astype('category')
    

preds = catboost_cl.predict_proba(sub[FEATS])[:,1]
    
s = pd.read_csv('output/submission.csv')
m = preds
s['prediction'] = m

s.to_csv('output/submission_cat_10000.csv', index=False)

In [48]:
def custom_train_test_split_fold_5(df, ratio=0.2, split=True):
    
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data, k = 0, 0
    user_ids =[[],[],[],[],[]]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            k += 1
            max_train_data_len += ratio*len(df)
        user_ids[k].append(user_id)

    return user_ids


In [55]:
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np

user_id = custom_train_test_split_fold_5(df)

for i in tqdm(range(5)):
    u = []
    for j in range(5):
        if j != i:
            u += user_id[j]
        
    train = df[df['userID'].isin(u)]
    test = df[df['userID'].isin(u) == False]

    #test데이터셋은 각 유저의 마지막 interaction만 추출
    test = test[test['userID'] != test['userID'].shift(-1)]
    
    y_train_cat = train['answerCode']
    X_train_cat = train.drop(['answerCode'], axis=1)[FEATS]

    y_test_cat = test['answerCode']
    X_test_cat = test.drop(['answerCode'], axis=1)[FEATS]

   
    # catboost_cl = CatBoostClassifier(cat_features=cate, n_estimators=5000, use_best_model=True,
    # eval_metric ='AUC')
    catboost_cl = CatBoostClassifier(cat_features=cate, n_estimators=5000, eval_metric ='AUC', 
                                     use_best_model=True, od_type = "Iter", od_wait = 100)
    
    catboost_cl.fit(X_train_cat, y_train_cat, verbose=500, eval_set=(X_test_cat, y_test_cat))
    preds = catboost_cl.predict_proba(X_test_cat)[:,1]
    acc = accuracy_score(y_test_cat, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_test_cat, preds)
    
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    
    catboost_cl.save_model(f'catboost_info/model/catboost_{i}')

  0%|          | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.103118
0:	test: 0.6934543	best: 0.6934543 (0)	total: 868ms	remaining: 1h 12m 20s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8048703488
bestIteration = 273

Shrink model to first 274 iterations.
VALID AUC : 0.8048703488455524 ACC : 0.7230769230769231



 20%|██        | 1/5 [04:58<19:52, 298.20s/it]

Learning rate set to 0.103122
0:	test: 0.7138384	best: 0.7138384 (0)	total: 771ms	remaining: 1h 4m 15s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8081097349
bestIteration = 219

Shrink model to first 220 iterations.


 40%|████      | 2/5 [09:11<14:14, 284.83s/it]

...

VALID AUC : 0.7934909084277142 ACC : 0.7191087103308575

Learning rate set to 0.103119
0:	test: 0.7250497	best: 0.7250497 (0)	total: 844ms	remaining: 1h 10m 20s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8075652673
bestIteration = 119

Shrink model to first 120 iterations.


 80%|████████  | 4/5 [15:53<03:59, 239.74s/it]

VALID AUC : 0.8075652673294167 ACC : 0.7318212141427618

Learning rate set to 0.103109
0:	test: 0.6907348	best: 0.6907348 (0)	total: 843ms	remaining: 1h 10m 13s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7925107527
bestIteration = 217

Shrink model to first 218 iterations.


100%|██████████| 5/5 [20:07<00:00, 241.51s/it]

VALID AUC : 0.792510752688172 ACC : 0.7163879598662207



In [ ]:
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import StratifiedKFold

str_kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

X, y = df[FEATS], df['answerCode']

for i, (train_index, test_index) in tqdm(enumerate(str_kf.split(X, y))):
    X_train_cat, X_test_cat = X.loc[train_index], X.loc[test_index]
    y_train_cat, y_test_cat = y.loc[train_index], y.loc[test_index]

   
    catboost_cl = CatBoostClassifier(cat_features=cate, n_estimators=10000, use_best_model=True,
    eval_metric ='AUC', od_type = "Iter", od_wait = 100)

    
    catboost_cl.fit(X_train_cat, y_train_cat, verbose=500, eval_set=(X_test_cat, y_test_cat))
    preds = catboost_cl.predict_proba(X_test_cat)[:,1]
    acc = accuracy_score(y_test_cat, np.where(preds >= 0.5, 1, 0))
    auc = roc_auc_score(y_test_cat, preds)
    
    print(f'VALID AUC : {auc} ACC : {acc}\n')
    
    catboost_cl.save_model(f'catboost_info/model/catboost_{i}')

0it [00:00, ?it/s]

Learning rate set to 0.076273
0:	test: 0.7810810	best: 0.7810810 (0)	total: 1.87s	remaining: 5h 11m 11s
500:	test: 0.8399595	best: 0.8399595 (500)	total: 15m 28s	remaining: 4h 53m 17s
1000:	test: 0.8409357	best: 0.8409357 (1000)	total: 30m 19s	remaining: 4h 32m 41s
1500:	test: 0.8415582	best: 0.8415582 (1500)	total: 45m 24s	remaining: 4h 17m 6s
2000:	test: 0.8419653	best: 0.8419653 (2000)	total: 1h 31s	remaining: 4h 1m 56s
2500:	test: 0.8421943	best: 0.8421961 (2483)	total: 1h 15m 46s	remaining: 3h 47m 13s
3000:	test: 0.8424450	best: 0.8424465 (2987)	total: 1h 32m 58s	remaining: 3h 36m 49s
3500:	test: 0.8425199	best: 0.8425220 (3483)	total: 1h 49m 57s	remaining: 3h 24m 6s
4000:	test: 0.8426113	best: 0.8426113 (4000)	total: 2h 7m 16s	remaining: 3h 10m 50s
4500:	test: 0.8427666	best: 0.8427688 (4489)	total: 2h 24m 19s	remaining: 2h 56m 19s
5000:	test: 0.8429928	best: 0.8429942 (4991)	total: 2h 40m 56s	remaining: 2h 40m 52s
5500:	test: 0.8430568	best: 0.8430590 (5453)	total: 2h 57m 21s	re

1it [3:24:34, 12274.39s/it]

VALID AUC : 0.8431332107831392 ACC : 0.7909863972509462

Learning rate set to 0.076273
0:	test: 0.7823429	best: 0.7823429 (0)	total: 2.2s	remaining: 6h 6m 26s
500:	test: 0.8396957	best: 0.8396957 (500)	total: 18m 1s	remaining: 5h 41m 42s
1000:	test: 0.8405531	best: 0.8405531 (999)	total: 33m 7s	remaining: 4h 57m 47s
1500:	test: 0.8410104	best: 0.8410129 (1487)	total: 48m 5s	remaining: 4h 32m 19s
2000:	test: 0.8412819	best: 0.8412824 (1998)	total: 1h 3m 21s	remaining: 4h 13m 14s
2500:	test: 0.8414930	best: 0.8414930 (2500)	total: 1h 18m 43s	remaining: 3h 56m 2s
3000:	test: 0.8417015	best: 0.8417027 (2955)	total: 1h 34m 16s	remaining: 3h 39m 51s
3500:	test: 0.8418839	best: 0.8418840 (3499)	total: 1h 49m 47s	remaining: 3h 23m 48s
4000:	test: 0.8419702	best: 0.8419718 (3988)	total: 2h 5m 52s	remaining: 3h 8m 44s
4500:	test: 0.8421065	best: 0.8421069 (4498)	total: 2h 21m 51s	remaining: 2h 53m 19s
5000:	test: 0.8422781	best: 0.8422807 (4978)	total: 2h 38m 16s	remaining: 2h 38m 12s
5500:	test

In [15]:
sub = pd.read_csv('../../data/infer.csv')

for i in cate:
    sub[i] = sub[i].astype('category')
    
p = []

for i in tqdm(range(5)):
    catboost_cl = CatBoostClassifier(cat_features=cate, n_estimators=5000)
    catboost_cl.load_model(f'catboost_info/model/catboost_{i}')
    preds = catboost_cl.predict_proba(sub[FEATS])[:,1]
    p.append(preds)
    
s = pd.read_csv('output/submission.csv')
m = (p[0] + p[1] + p[2] + p[3] + p[4])/5
s['prediction'] = m

s.to_csv('output/submission_cat_elo.csv', index=False)

100%|██████████| 5/5 [00:13<00:00,  2.65s/it]


In [2]:
sub = pd.read_csv('../../data/infer.csv')

for i in cate:
    sub[i] = sub[i].astype('category')
    
p = []

i=0
catboost_cl = CatBoostClassifier(cat_features=cate, n_estimators=5000)
catboost_cl.load_model(f'catboost_info/model/catboost_{i}')
preds = catboost_cl.predict_proba(sub[FEATS])[:,1]
p.append(preds)
    
s = pd.read_csv('output/submission.csv')
m = p[0]
s['prediction'] = m

s.to_csv('output/test.csv', index=False)

In [19]:
train = pd.read_csv('../../data/infer.csv')

train['as0_as1'] =  [str(i)+'_'+str(j) for i, j in zip(list(train.assessmentItemID0), list(train.assessmentItemID1))]
train['as0_as2'] =  [str(i)+'_'+str(j) for i, j in zip(list(train.assessmentItemID0), list(train.assessmentItemID2))]
train['as1_as2'] =  [str(i)+'_'+str(j) for i, j in zip(list(train.assessmentItemID1), list(train.assessmentItemID2))]

cate = ['as0_as1', 'as0_as2', 'as1_as2']
for i in cate:
    cate2label = {j:i for i,j in enumerate(train[i].unique())}
    train[i] = train[i].map(cate2label)

train.to_csv('../../data/infer.csv', index=False)

In [6]:
test = pd.read_csv('../../data/test.csv')
sub = test[test['answerCode'] == -1]
sub['userID'] = sub['userID'].map(cate2label)

import time
from datetime import datetime
def convert_time(s):
    timestamp = time.mktime(
        datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple()
    )
    return int(timestamp)

sub['Timestamp'] = sub['Timestamp'].apply(convert_time)

preds = catboost_cl.predict_proba(sub[FEATS])[:,1]

s = pd.read_csv('output/submission.csv')
s['prediction'] = preds

s.to_csv('output/submission_cat_2.0.csv', index=False)

/tmp/ipykernel_16216/3616887153.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['userID'] = sub['userID'].map(cate2label)
/tmp/ipykernel_16216/3616887153.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['Timestamp'] = sub['Timestamp'].apply(convert_time)


In [3]:
test = pd.read_csv('../../data/test.csv')
test.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'month', 'hour', 'week', 'elapsed', 'elapsed_cate',
       'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2',
       'testId0', 'testId1', 'test0_mean', 'test0_std', 'test1_mean',
       'test1_std', 'tag_mean', 'tag_std', 'ass0_mean', 'ass0_std',
       'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std', 'user_correct_answer',
       'user_total_answer', 'user_acc', 'recAccuracy', 'recCount'],
      dtype='object')

In [ ]:
FEATS = ['Timestamp', 'userID',
       'KnowledgeTag', 'assessmentItemID0', 'assessmentItemID1',
       'assessmentItemID2', 'month', 'hour', 'week', 'elapsed', 'elapsed_cate',
       'tag_mean', 'tag_std', 'ass0_mean', 'ass0_std', 'ass1_mean', 'ass1_std',
       'ass2_mean', 'ass2_std', 'user_total_answer']

In [13]:
a = pd.read_csv('output/cat_0.8297_0.7823.csv')
b = pd.read_csv('output/lgbm_8200_7527.csv')
c = pd.read_csv('output/gcn_7858_7177.csv')

d = (a['prediction']*0.45 + b['prediction']*0.4 + c['prediction']*0.15)
b['prediction'] = d
b.to_csv('output/p2.csv', index=False)